<a href="https://colab.research.google.com/github/TissaMaria/AI2024/blob/main/FOL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from itertools import product

class ForwardChaining:
    def __init__(self, rules, facts):
        self.rules = rules  # List of rules in the form of (conditions, conclusion)
        self.facts = set(facts)  # Set of known facts
        self.derived_facts = {}  # To store proof tree (inferred facts)

    def infer(self, query):
        """
        Perform forward chaining to infer new facts until query is proved or no new facts.
        """
        while True:
            new_facts = set()

            # Iterate over all rules
            for conditions, conclusion in self.rules:
                substitutions = self.check_conditions(conditions)
                for sub in substitutions:
                    inferred_fact = self.substitute(sub, conclusion)

                    # Add to new facts if not already known
                    if inferred_fact not in self.facts and inferred_fact not in new_facts:
                        new_facts.add(inferred_fact)
                        self.derived_facts[inferred_fact] = (conditions, sub)

            # Check if query is satisfied
            if query in new_facts:
                return True

            # Stop if no new facts can be inferred
            if not new_facts:
                return False

            # Add new facts to the knowledge base
            self.facts.update(new_facts)

    def check_conditions(self, conditions):
        """
        Check if conditions can be satisfied by existing facts.
        Returns possible substitutions that make the conditions true.
        """
        possible_substitutions = []

        for condition in conditions:
            matching_facts = []
            for fact in self.facts:
                sub = self.unify(condition, fact)
                if sub is not None:
                    matching_facts.append(sub)
            possible_substitutions.append(matching_facts)

        # Generate all combinations of substitutions for all conditions
        substitutions = []
        for combination in product(*possible_substitutions):
            combined_sub = {}
            valid = True
            for sub in combination:
                for key in sub:
                    if key in combined_sub and combined_sub[key] != sub[key]:
                        valid = False  # Conflict in substitutions
                        break
                    combined_sub[key] = sub[key]
                if not valid:
                    break
            if valid:
                substitutions.append(combined_sub)

        return substitutions

    def unify(self, condition, fact):
        """
        Unify a condition with a fact, returning a substitution if successful.
        """
        condition_predicate, condition_args = self.parse_predicate(condition)
        fact_predicate, fact_args = self.parse_predicate(fact)

        if condition_predicate != fact_predicate or len(condition_args) != len(fact_args):
            return None

        substitution = {}
        for c_arg, f_arg in zip(condition_args, fact_args):
            if c_arg.islower():  # Variable
                if c_arg in substitution and substitution[c_arg] != f_arg:
                    return None  # Conflict in substitution
                substitution[c_arg] = f_arg
            elif c_arg != f_arg:  # Constant mismatch
                return None

        return substitution

    def substitute(self, substitution, conclusion):
        """
        Apply a substitution to a conclusion to generate a fact.
        """
        predicate, args = self.parse_predicate(conclusion)
        substituted_args = [substitution.get(arg, arg) for arg in args]
        return f"{predicate}({', '.join(substituted_args)})"

    def parse_predicate(self, expression):
        """
        Parse a predicate into its name and arguments.
        Example: "Parent(x, y)" -> ("Parent", ["x", "y"])
        """
        predicate_name = expression.split('(')[0]
        arguments = expression.split('(')[1].strip(')').split(', ')
        return predicate_name, arguments


# Knowledge Base (Rules and Facts)
rules = [
    (["Parent(x, y)", "Parent(y, z)"], "Grandparent(x, z)"),  # Rule: Parent(x, y) AND Parent(y, z) => Grandparent(x, z)
]
facts = [
    "Parent(John, Mary)",
    "Parent(Mary, Alice)",
]

# Query
query = "Grandparent(John, Alice)"

# Forward Chaining
fc = ForwardChaining(rules, facts)
result = fc.infer(query)

# Output Results
if result:
    print(f"The query '{query}' is TRUE.")
    print("Proof Tree:")
    for fact, (conditions, sub) in fc.derived_facts.items():
        print(f"{fact} was inferred from {conditions} using substitution {sub}.")
else:
    print(f"The query '{query}' is FALSE.")


The query 'Grandparent(John, Alice)' is TRUE.
Proof Tree:
Grandparent(John, Alice) was inferred from ['Parent(x, y)', 'Parent(y, z)'] using substitution {'x': 'John', 'y': 'Mary', 'z': 'Alice'}.
